In [1]:
!pip install cbastats
!pip install pymysql

In [1]:
from cbastats.Team import *
from cbastats.Player import *
from sqlalchemy import create_engine
import datetime

In [2]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-08-08 01:55:15.035331


# 载入数据

In [14]:
user_name = 'guest'
passcode = 'Guest123456'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

connection= engine.connect()

df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection);

connection.close()

# 所有球队高阶数据统计

In [7]:
teams = Team('',df)

teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)

teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df.round(1)

# 所有球员高阶数据

In a later chapter of Basketball on Paper, Oliver emphasized that Offensive Ratings shouldn't be viewed in a vacuum. Introducing a concept he called "Skill Curves", he acknowledged that a player's ORtg needed to be judged in conjunction with his Usage Rate, a measure of how big a role the player fills in his team's offense. The bigger the role, the more difficult it is to maintain a high ORtg; the smaller the role, the easier it is to be highly efficient. Because of this, Oliver stressed that a player's ORtg should primarily be compared to those of other players in a similar role.

Out of necessity (owing to a lack of defensive data in the basic boxscore), individual Defensive Ratings are heavily influenced by the team's defensive efficiency. They assume that all teammates are equally good (per minute) at forcing non-steal turnovers and non-block misses, as well as assuming that all teammates face the same number of total possessions per minute.

In [11]:
players = Player('',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)


players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df.reset_index(inplace=True)
players_df.sort_values(by=['球队','NetRating','UsagePercent'],ascending=False,inplace=True)
players_df

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
272,约翰逊,青岛,131.671544,118.351276,13.320269,27.961744,893,1,138,283,...,36,251,687,240,382,0,4,207,299,29
38,刘传兴,青岛,119.745393,118.391091,1.354301,18.614538,776,29,102,190,...,18,118,349,125,215,0,0,99,141,44
12,亚当斯,青岛,115.163768,116.408521,-1.244753,39.500900,1036,28,27,209,...,1,223,1012,214,379,122,346,218,249,29
243,王庆明,青岛,113.310661,119.541128,-6.230467,15.724908,1121,26,23,116,...,3,93,424,53,129,86,199,60,79,47
324,赵戌宏,青岛,112.180791,119.434821,-7.254031,14.983049,351,14,38,71,...,8,30,120,48,78,0,0,24,39,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,吴永盛,上海,81.991858,117.250759,-35.258901,18.348755,51,2,1,2,...,0,4,16,7,11,0,5,2,2,7
112,孟令源,上海,NaN,117.250759,NaN,28.999468,17,2,0,0,...,0,1,3,0,6,1,4,0,0,6
88,唐子豪,上海,NaN,124.023135,NaN,16.599022,27,3,3,1,...,0,2,10,2,3,2,5,0,0,5
185,朱瀛,上海,NaN,111.736369,NaN,5.432407,33,1,3,2,...,0,2,2,1,3,0,0,0,0,5


In [17]:
players_df.to_csv(players_stats_output,index=False)

# 单个球队高阶数据统计

In [19]:
teams = Team('广东',df)

teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)
teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
广东,16.966667,97.371742,127.174818,109.288114,17.886704


# 单个球员高阶数据

In [8]:
players = Player('易建联',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)

players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df

,,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
球员,球队,,,,,,,,,,,,,,,,,,,,,
易建联,广东,132.364085,110.994395,21.36969,24.183248,992,33,89,261,45,62,...,73,183,675,192,335,41,104,168,209,33
